# Fit Neg Binomial Variance Component model 

Taking inspiration from [GLM.jl](https://github.com/JuliaStats/GLM.jl/blob/master/src/negbinfit.jl#L68), we will:
+ Initialize `r` with Poisson regression fit
+ Perform block updates:
    - Fix $r$, fit negative binomial copula for 10 iterations
    - Fix $\beta$ and AR parameters, fit $r$ using Newton for 10 iterations
    - Repeat until convergence

# Block updates

In [3]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!

# Random.seed!(1234)
Random.seed!(12345)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.1
variance_component_2 = 0.5

r = 100
p = 0.7
μ = r * (1-p) * inv(p)

# var = r * (1-p) * inv(p^2)

# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Vector{Vector{Float64}}:
 [49.0, 55.0, 38.0, 39.0, 38.0]
 [38.0, 30.0, 45.0, 27.0, 42.0]
 [49.0, 63.0, 49.0, 43.0, 35.0]
 [50.0, 40.0, 35.0, 61.0, 38.0]
 [27.0, 52.0, 51.0, 42.0, 39.0]
 [42.0, 47.0, 39.0, 68.0, 36.0]
 [33.0, 52.0, 38.0, 51.0, 50.0]
 [50.0, 44.0, 39.0, 47.0, 56.0]
 [49.0, 53.0, 56.0, 56.0, 48.0]
 [43.0, 44.0, 54.0, 45.0, 36.0]
 [44.0, 36.0, 34.0, 42.0, 29.0]
 [48.0, 37.0, 40.0, 35.0, 49.0]
 [44.0, 39.0, 43.0, 43.0, 33.0]
 ⋮
 [40.0, 31.0, 46.0, 47.0, 51.0]
 [38.0, 53.0, 58.0, 40.0, 36.0]
 [44.0, 53.0, 37.0, 57.0, 30.0]
 [36.0, 33.0, 40.0, 43.0, 37.0]
 [44.0, 35.0, 28.0, 35.0, 42.0]
 [42.0, 26.0, 51.0, 34.0, 42.0]
 [42.0, 33.0, 47.0, 36.0, 36.0]
 [43.0, 49.0, 37.0, 48.0, 55.0]
 [37.0, 32.0, 27.0, 34.0, 46.0]
 [50.0, 43.0, 38.0, 32.0, 32.0]
 [45.0, 43.0, 54.0, 47.0, 42.0]
 [41.0, 38.0, 38.0, 36.0, 63.0]

In [4]:
d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{NBCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = NBCopulaVCObs(y, X, V, d, link)
end
gcm = NBCopulaVCModel(gcs);
# gcm.r[1] = r

In [5]:
initialize_model!(gcm)
@show gcm.β
# @show gcm.Σ
@show gcm.r

Initializing NegBin r to Poisson regression values
initializing β using Newton's Algorithm under Independence Assumption
gcm.β = [3.7627017783313135]
initializing variance components using MM-Algorithm
gcm.Σ = [1.0, 1.0]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

initializing variance components using MM-Algorithm

┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_new.jl:31



gcm.β = [3.7619029555583485]
gcm.r = [65.09217455072616]


1-element Vector{Float64}:
 65.09217455072616

In [6]:
β_true

3.757872325600888

In [7]:
GLMCopula.loglikelihood!(gcm, true, true)

-178754.89002641922

In [8]:
# default is Quasi-Newton
@time GLMCopula.fit!(gcm, maxBlockIter=100)

Initializing NegBin r to Poisson regression values
initializing β using Newton's Algorithm under Independence Assumption
gcm.β = [3.7627017783313135]
initializing variance components using MM-Algorithm
gcm.Σ = [1.0, 1.0]


┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_new.jl:31


initializing variance components using MM-Algorithm
Converging when tol ≤ 1.0e-6 (max block iter = 100)
Block iter 1 r = 102.62, logl = -178727.85, tol = 178727.85020508055
Block iter 2 r = 101.59, logl = -178727.44, tol = 2.295589225579987e-6
Block iter 3 r = 100.8, logl = -178727.2, tol = 1.3659562765417012e-6
  8.694038 seconds (7.21 M allocations: 1.029 GiB, 2.87% gc time, 4.12% compilation time)


-178726.9931765957

In [17]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.7577367229152956; true β = 3.757872325600888
estimated variance component 1 = 0.08454686818715344; true variance component 1 = 0.1
estimated variance component 2 = 0.3859156499415465; true variance component 2 = 0.5
estimated r = 96.0785202732631; true r = 100


In [7]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.7577367229152956; true β = 3.757872325600888
estimated variance component 1 = 0.08454686818715344; true variance component 1 = 0.1
estimated variance component 2 = 0.3859156499415465; true variance component 2 = 0.5
estimated r = 96.0785202732631; true r = 100


In [7]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 1.454827543795997; true β = 1.4552872326068422
estimated variance component 1 = 0.10456366108069844; true variance component 1 = 0.1
estimated variance component 2 = 0.44579361811367024; true variance component 2 = 0.5
estimated r = 9.969213381836406; true r = 10


# Benchmark/profile code

In [191]:
using BenchmarkTools, SpecialFunctions

function update_HΣ_test!(gcm)
    T = Float64
    fill!(gcm.HΣ, 0.0)
    @inbounds for j in 1:gcm.m
        @simd for i in 1:length(gcm.storage_n)
            gcm.hess1[j, i] = gcm.QF[i, j] * gcm.storage_n[i]
            gcm.hess2[j, i] = gcm.TR[i, j] * gcm.storage_n2[i]
        end
    end
    BLAS.gemm!('N', 'T', -T(1), gcm.hess1, gcm.hess1, T(0), gcm.HΣ)
    BLAS.gemm!('N', 'T', T(1), gcm.hess2, gcm.hess2, T(1), gcm.HΣ)
end

update_HΣ_test! (generic function with 1 method)

In [192]:
gc = gcm.data[1]
β = gcm.β
@btime update_HΣ_test!($gcm)

  65.226 μs (0 allocations: 0 bytes)


2×2 Matrix{Float64}:
 23863.7  23863.7
 23863.7  23863.7

# Let's see how to optimally block update 

## 1 iteration each

In [58]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.7640471229274333; true β = 3.757872325600888
estimated variance component 1 = 0.010091096783959048; true variance component 1 = 0.1
estimated variance component 2 = 0.010061528161806528; true variance component 2 = 0.5
estimated r = 59.22108970817275; true r = 100


## 5 iteration each

In [45]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.761120679347187; true β = 3.757872325600888
estimated variance component 1 = 0.04200544450800426; true variance component 1 = 0.1
estimated variance component 2 = 0.020655323963911138; true variance component 2 = 0.5
estimated r = 64.2226158070482; true r = 100


## 10 iteration each

In [51]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.7577367229152965; true β = 3.757872325600888
estimated variance component 1 = 0.08454686818713725; true variance component 1 = 0.1
estimated variance component 2 = 0.3859156499408988; true variance component 2 = 0.5
estimated r = 96.07852027325903; true r = 100


## 1 IPOPT iter + complete Newton fit

In [65]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.762274167394871; true β = 3.757872325600888
estimated variance component 1 = 0.044468809218469255; true variance component 1 = 0.1
estimated variance component 2 = 0.046354142614646404; true variance component 2 = 0.5
estimated r = 67.38079696512439; true r = 100


## 5 IPOPT iter + complete Newton fit

In [72]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.759537350779348; true β = 3.757872325600888
estimated variance component 1 = 0.05312813376763176; true variance component 1 = 0.1
estimated variance component 2 = 0.07266779547787298; true variance component 2 = 0.5
estimated r = 70.93622181411288; true r = 100


## 10 IPOPT iter + complete Newton fit (this reached max iter count)

In [79]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.757598085312234; true β = 3.757872325600888
estimated variance component 1 = 0.08847643797375318; true variance component 1 = 0.1
estimated variance component 2 = 0.418885780706873; true variance component 2 = 0.5
estimated r = 98.01186301721411; true r = 100


## Copying GLM.jl

In [7]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 1.456226847936331; true β = 1.4552872326068422
estimated variance component 1 = 0.10133551134062899; true variance component 1 = 0.1
estimated variance component 2 = 0.4371874111348784; true variance component 2 = 0.5
estimated r = 9.69061414063922; true r = 10


In [73]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.758435534322703; true β = 3.757872325600888
estimated variance component 1 = 0.0934850410967605; true variance component 1 = 0.1
estimated variance component 2 = 0.3827355454688751; true variance component 2 = 0.5
estimated r = 94.45211633725886; true r = 100
